In [ ]:
#selloutData.describe()
#-qty정제 다른 방법
#def qtyReplace(inValue):
  #  if inValue < 0 : outValue = 0
  #  else : outValue = inValue
  #  return outValue

#selloutData["QTY_NEW"] = selloutData.QTY.apply(qtyReplace)

#selloutData.loc[ selloutData.QTY < 0, "QTY_NEW"] = 0
#selloutData.loc[ selloutData.QTY >= 0, "QTY_NEW"] = selloutData.QTY


라이브러리 불러오기 / 파일 읽어와 형변환 

In [36]:
#라이브러리 불러오기
import pandas as pd
import numpy as np

#파일 불러오기 및 형 변환
selloutData = pd.read_csv("./dataset/kopo_channel_seasonality_new.csv")
selloutData = selloutData.astype({"REGIONID":str,
                                  "PRODUCT":str,
                                  "YEARWEEK":str,
                                  "QTY":float})

In [37]:
#if QTY_NEW < 0 : 0 , else: Qty값 
selloutData["QTY_NEW"] = np.where( selloutData.QTY < 0,0,selloutData["QTY"] )

#YEARWEEK.str로 문자열 인덱스로 활용 YEAR / WEEK 정의 
selloutData["YEAR"] , selloutData["WEEK"] = selloutData.YEARWEEK.str[:-2] , selloutData.YEARWEEK.str[-2:]

#정의한 WEEK가 52이하인 값을 조회해 변수에 대입  /selloutData.WEEK.max()값을 얻고 isin을 활용해도 좋을 듯 하다.
refinedSelloutData = selloutData.loc[ selloutData.WEEK.astype(int) <=52 ]

In [38]:
refinedSelloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,23.0,2014,02
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


In [39]:
#sortKey/sort code
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

sortedData = refinedSelloutData.sort_values(by=sortKey) 
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
298,A00,PRODUCT34,201401,661.0,661.0,2014,01
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03
205,A00,PRODUCT34,201404,532.0,532.0,2014,04
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05
...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


In [40]:
#group by code
groupKey = ["REGIONID","PRODUCT","YEAR"] 

#groupby groupkey 그룹별 qty_new값의 평균을 나타soa / restart index , inplace
groupData = sortedData.groupby(groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
#mean컬럼 -> qty_new로 재설정 
groupData = groupData.rename(columns = {"mean":"QTY_MEAN"})

In [41]:
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [42]:
sortedData.groupby(groupKey)["QTY"].describe() #숫자컬럼 기본 통계데이터 

count         mean          std     min      25%  \
REGIONID PRODUCT   YEAR                                                     
A00      PRODUCT34 2014   52.0   275.961538   162.179634    60.0   149.50   
                   2015   52.0    86.634615    73.246034    24.0    41.50   
                   2016   52.0    36.576923    94.897539     2.0     7.00   
         PRODUCT58 2014   52.0     2.673077     3.889257     0.0     0.00   
                   2015   52.0     5.711538     6.675494     1.0     2.00   
...                        ...          ...          ...     ...      ...   
A77      PRODUCT1  2015   52.0  3030.019231   846.496783  1882.0  2487.00   
                   2016   52.0  3375.326923   702.359238  1995.0  2918.00   
         PRODUCT12 2014   52.0  2035.788462   762.881491   739.0  1551.50   
                   2015   52.0  3540.980769   852.593402  2314.0  2794.25   
                   2016   52.0  4837.153846  1841.810824  2780.0  3738.75   

                            50%      75%      max  
REGIONID PRODUCT   YEAR                            
A00      PRODUCT34 2014   232.0   365.25    679.0  
                   2015    65.0   109.00    479.0  
                   2016    12.5    22.00    561.0  
         PRODUCT58 2014     1.0     3.00     16.0  
                   2015     3.5     6.00     39.0  
...                         ...      ...      ...  
A77      PRODUCT1  2015  2889.5  3342.25   6859.0  
                   2016  3121.5  3700.75   5040.0  
         PRODUCT12 2014  1990.0  2403.00   3916.0  
                   2015  3500.0  4104.25   5601.0  
                   2016  4306.0  5136.25  14734.0  

[2382 rows x 8 columns]

In [43]:
#이평선 함수 Framename["colname"].rolling(window = 5,   -> 나를 기준으로 날개를 몇개 펼칠지 정의 (여기서는 좌2 우2)
#                                        center=True,  ->중간값을 기준으로 평균을 내라 
#                                        min_periods = 1 ->window 에서 앞에 값이 없어 날개를 펼칠 수 없으면 null값을 집어넣음. 이 부분은 null이아닌 값이 하나라도 있으면 평균을 구함 ).mean() ->평균내라

In [44]:
sortedData["MAD5"]=sortedData.QTY.rolling(window = 5,
                      min_periods=1,
                      center = True).mean()
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MAD5
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,639.333333
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,612.500000
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,593.200000
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,545.600000
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05,491.200000
...,...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000


In [45]:
#옵션에 따른 변화 확인 
sortedData["MAD5_basic"]=sortedData.QTY.rolling(window = 5,
                                                center = False).mean()

sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MAD5,MAD5_basic
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,639.333333,NaN
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,612.500000,NaN
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,593.200000,NaN
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,545.600000,NaN
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05,491.200000,593.2
...,...,...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000,4119.2
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000,4314.6
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000,4582.6
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000,4718.2


In [46]:
sortedData.drop(columns=["MAD5_basic"])

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MAD5
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,639.333333
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,612.500000
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,593.200000
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,545.600000
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05,491.200000
...,...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000


In [49]:
sortedData["EFFECT"] = sortedData.QTY / sortedData.MAD5   #추세선 대비 실제판매량 
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MAD5,MAD5_basic,EFFECT
298,A00,PRODUCT34,201401,661.0,661.0,2014,01,639.333333,NaN,1.033889
1757,A00,PRODUCT34,201402,679.0,679.0,2014,02,612.500000,NaN,1.108571
3125,A00,PRODUCT34,201403,578.0,578.0,2014,03,593.200000,NaN,0.974376
205,A00,PRODUCT34,201404,532.0,532.0,2014,04,545.600000,NaN,0.975073
4369,A00,PRODUCT34,201405,516.0,516.0,2014,05,491.200000,593.2,1.050489
...,...,...,...,...,...,...,...,...,...,...
102292,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000,4119.2,0.906036
97525,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000,4314.6,1.077953
102329,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000,4582.6,1.060287
101211,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000,4718.2,0.842672


In [50]:
sortedData.YEAR.drop_duplicates() #중복제거  오라클에서 distinct

298      2014
40331    2015
83502    2016
Name: YEAR, dtype: object

In [52]:
#3개 연도의 주차별 효과를 뽑아내고싶다.
groupKey = ["REGIONID","PRODUCT","WEEK"]
ratioData = sortedData.groupby(groupKey)[["EFFECT"]].agg("mean").reset_index()

In [54]:
ratioData.to_csv("./middleResult.csv",index=False,encoding="ms949")